In [ ]:
%pylab inline

# Adama example for DesignSafe-CI

![DesignSafe Logo](designsafe.jpg)

This is an example of building an Adama service.  We use the Haiti Earthquake Database and we construct a couple of web services from the data hosted at https://nees.org/dataview/spreadsheet/haiti.

## Setting up

The code for these services is in the directory `demo`:

In [ ]:
cd demo

We will construct two web services that will return the data in JSON format:

- `haiti`: will allow to query the database by building,
- `haiti_images`: will allow to retrieve the set of images for each building.

Each Adama service consists in two pieces of information:

- the metadata that describes the service,
- and the actual code.

This is an example of the metadata for the `haiti` service:

In [ ]:
%load services/haiti/metadata.yml

The code for the service looks very simple:

In [ ]:
%load services/haiti/main.py

## Interacting with Adama

To interact with Adama, we create an object with our credentials:

In [ ]:
import adamalib

TOKEN = 'My Token'                                                 # here goes your credentials
adama = adamalib.Adama('https://api.araport.org/community/v0.3',   # we use Araport for this example
                       token=TOKEN)                                

Now the `adama` object is connected to the server.  We can check that the server is up:

In [ ]:
adama.status

## Testing the new services locally

The services we are going to register in Adama can be tested first locally:

In [ ]:
import services.haiti.main

services.haiti.main.search({'building': 'A001'}, adama)

## Registering the services in Adama (skip this if using already registered ones)

Now we are ready to register the services in Adama.  We'll use the namespace `walter-dev` for testing purposes:

In [ ]:
adama['walter-dev'].services

To register a service, we just import its code and add it to the previous list (it may take a minute or two):

In [ ]:
import services.haiti.main            # the haiti service
import services.haiti_images.main     # the haiti_images service

haiti = adama['walter-dev'].services.add(services.haiti.main)
haiti_images = adama['walter-dev'].services.add(services.haiti_images.main)

haiti, haiti_images

The services are registered and can be accessed in https://araport.org.

## Using the services

Now that the services are registered in Adama, and the Python objects `haiti` and `haiti_images` are connected to the **remote** services, we can use them as regular objects.

In [ ]:
haiti = adama['walter-dev'].haiti
haiti_images = adama['walter-dev'].haiti_images

Let's test the services.  Note that now the code will be executed remotely in the Adama server:

In [ ]:
haiti.search(building='A001')

We can display an image using the `haiti_images` service:

In [ ]:
from IPython.display import Image

Image(data=haiti_images.search(building='A001', image=1).content)

Let's display the table of buildings together with their geographical coordinates and the priority index:

In [ ]:
from pandas import DataFrame

full = haiti.list()
columns = ['Building', 'Latitude', 'Longitude', 'Priority Index  [%]']
buildings = DataFrame([[row[col] for col in columns] for row in full if row['Latitude']],
                      columns=columns).convert_objects(convert_numeric=True)
buildings

We can operate on the data, for example doing a histogram of the priority index:

In [ ]:
from IPython.utils.path import get_ipython_dir
get_ipython_dir()

In [ ]:
plt.hist(buildings['Priority Index  [%]'], bins=50);

Let's display the buildings in a map, and let's load the images on demand:

In [ ]:
import services.map_tools
map_init()

In [ ]:
map_display(DataFrame.as_matrix(buildings)[:,:3],
            token=TOKEN)